In [1]:
import torch
import torch.nn as nn

In [2]:
class SimpleMlp(nn.Module):
    """
    MLP
    """
    def __init__(
        self,
        vec_length: int=16,
        hidden_unit_1: int=8,
        hidden_unit_2: int=2):
        """
        引数：
            vec_length：入力ベクトルの長さ
            hidden_unit_1：1つ目の線形層のニューロン層
            hidden_unit_2：2つ目の線形層のニューロン層
        """
        super(SimpleMlp, self).__init__()

        self.layer1 = nn.Linear(vec_length, hidden_unit_1)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_unit_1, hidden_unit_2)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        順伝播は線形層→ReLU→線形層の順番
        引数：
            x： 入力（B, D_in）
                B：バッチサイズ、D_in：ベクトルの長さ
        返り値：
            out： 出力（B, D_out）
                B：バッチサイズ、D_out：ベクトルの長さ
        """
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out

In [3]:
vec_length = 16
hidden_unit_1 = 8
hidden_unit_2 = 2

batch_size = 4

x = torch.randn(batch_size, vec_length)
net = SimpleMlp(vec_length, hidden_unit_1, hidden_unit_2)
out = net(x)
print(out.shape)

torch.Size([4, 2])


In [15]:
class ViTInputLayer(nn.Module):
    def __init__(
        self,
        in_channel:int=3,
        emb_dim:int=384,
        num_patch_row:int=2,
        image_size:int=32
    ):
        """
        引数：
            in_channel：入力画像のチャンネル数
            emb_dim：畳み込み後のベクトルの長さ
            num_path_row：高さ方向のパッチの数。例は2×2であるため、2をデフォルト値とした
            image_size:：入力画像の1辺の長さ、入力画像の高さと幅は同じであると仮定した
        """
        super(ViTInputLayer, self).__init__()
        self.in_channels = in_channel
        self.emb_dim = emb_dim
        self.num_patch_row = num_patch_row
        self.image_size = image_size

        # パッチの数
        # 例：入力画像を2×2のパッチに分ける場合、num_patchは
        self.num_patch = self.num_patch_row ** 2

        # パッチの大きさ
        # 例：入力画像の1辺の大きさが32の場合、patch_sizeは16
        self.patch_size = int(self.image_size // self.num_patch_row)

        # 入力画像のパッチへの分割＆パッチの埋め込みを一気に行う
        self.patch_emb_layer = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.emb_dim,
            kernel_size=self.patch_size,
            stride=self.patch_size
        )

        # クラストークン
        self.cls_token = nn.Parameter(
            torch.randn(1, 1, emb_dim)
        )

        # 位置埋め込み
        # クラストークンが先頭に結合されているため、
        # 長さ emb_dimの埋め込みベクトルを（パッチ数＋1）個用意
        self.pos_emb = nn.Parameter(
            torch.randn(1, self.num_patch+1, emb_dim)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        引数：
            x：　入力画像、形状は（B、C、H、W）
        返り値：
            z_0：ViTへの入力、形状は（B、N、D）
        """
        # パッチの埋め込み＆flatten
        # パッチの埋め込み（B、C、H、W）->（B、D、H/P、W/P）
        # ここで、Pはパッチ1辺の長さ
        z_0 = self.patch_emb_layer(x)

        # パッチのflatten（B、D、H/P、W/P）->（B、D、Np）
        # ここで、Npはパッチの数（=H*W/P^2）
        z_0 = z_0.flatten(2)

        # 軸の入れ替え（B、D、Np）->（B、Np、D）
        z_0 = z_0.transpose(1, 2)

        # パッチの埋め込みの先頭にクラストークンを結合
        # （B、Np、D）->(B, N, D)
        # N = (Np + 1)であることに留意
        # また、cls_tokenの形状は(1, 1, D)であるため、
        # repeatメソッドによって(B, 1, D)に変換してからパッチの埋め込みとの結合を行う
        z_0 = torch.cat(
            [self.cls_token.repeat(repeats=(x.size(0), 1, 1)), z_0], dim=1
        )

        # 位置埋め込みの加算
        # (B, N, D)->(B, N, D)
        z_0 = z_0 + self.pos_emb

        return z_0

In [16]:
batch_size, channel, height, width = 2, 3, 32, 32
x = torch.randn(batch_size, channel, height, width)
input_layer = ViTInputLayer(num_patch_row=2)
z_0 = input_layer(x)

print(z_0.shape)

torch.Size([2, 5, 384])
